In [1]:
### 1.Chain异步API, 所有 以a开头的方法都是异步的，底层asyncio
# 当前LLMChain(其方法 arun, apredict, acall) 支持异步
# ChatVectorDBChain, and QA chains 也支持，其他的还在路上

In [4]:
### 2. __call__ && run 方法

# 所有继承Chain都有 __call__方法
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

chat = ChatOpenAI(temperature=0)
prompt_template = "Tell me a {adjective} joke"
llm_chain = LLMChain(llm=chat, prompt=PromptTemplate.from_template(prompt_template))

print(llm_chain(inputs={"adjective": "corny"}))

# 默认情况下，__call__ 返回输入键值和输出键值。可以将 return_only_outputs 设置为 True，以仅返回输出键值。

llm_chain("corny", return_only_outputs=True)

{'adjective': 'corny', 'text': 'Why did the tomato turn red? Because it saw the salad dressing!'}


{'text': 'Why did the tomato turn red? Because it saw the salad dressing!'}

In [7]:
# llm_chain only has one output key, so we can use run
print(llm_chain.output_keys)
# 如果链只输出一个输出键（即其output_keys中只有一个元素），则可以使用run方法。
# 请注意，run方法输出的是字符串而不是字典。
llm_chain.run({"adjective": "corny"})

['text']


'Why did the tomato turn red? Because it saw the salad dressing!'

In [8]:
# 如果只有一个输入键，则可以直接输入字符串，无需指定输入映射。

# These two are equivalent
print(llm_chain.run({"adjective": "corny"}))
print(llm_chain.run("corny"))

# These two are also equivalent
print(llm_chain("corny"))
print(llm_chain({"adjective": "corny"}))

Why did the tomato turn red? Because it saw the salad dressing!
Why did the tomato turn red? Because it saw the salad dressing!
{'adjective': 'corny', 'text': 'Why did the tomato turn red? Because it saw the salad dressing!'}
{'adjective': 'corny', 'text': 'Why did the tomato turn red? Because it saw the salad dressing!'}


In [10]:
### 3.custom chain
# 需要继承Chain并实现下面方法

from __future__ import annotations

from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class MyCustomChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    llm: BaseLanguageModel
    output_key: str = "text"  #: :meta private:

    class Config:
        """这个pydantic对象的配置."""

        extra = Extra.forbid            # private
        arbitrary_types_allowed = True  # 允许任意的类型

    @property
    def input_keys(self) -> List[str]:
        """将会是提示所期望的任何键。.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """将始终返回文本键。.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # 自定义链逻辑放在这里。
        
        # 这只是一个模仿LLMChain的示例
        prompt_value = self.prompt.format_prompt(**inputs)
        print(f'自定义chain: prompt_value={prompt_value}')

        # 每当调用一个语言模型或另一个链时，应该将回调管理器传递给它。
        # 这样可以通过在外部运行时注册的任何回调来跟踪内部运行。
        # 可以通过调用`run_manager.get_child()`来获取此回调管理器，如下所示。
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        
        print(f'自定义chain: response={response}')

        # 如果你想记录关于这次运行的一些信息，你可以通过调用`run_manager`上的方法来实现，
        # 如下所示。这将触发为该事件注册的任何回调函数。
        if run_manager:
            run_manager.on_text("Log something about this run")

        return {self.output_key: response.generations[0][0].text}

    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # 同上，只不过异步
        prompt_value = self.prompt.format_prompt(**inputs)

        response = await self.llm.agenerate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        
        # response 格式：
        # generations=[[ChatGeneration(
        #       text='response xxx', 
        #       generation_info=None, 
        #       message=AIMessage(content='response xxx', additional_kwargs={}, example=False))]]
        # llm_output={'token_usage': {'prompt_tokens': 14, 'completion_tokens': 15, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo'} 
        # run=RunInfo(run_id=UUID('224a4b3b-89c3-43fa-816e-3c78441674e5'))

        if run_manager:
            await run_manager.on_text("Log something about this run")

        return {self.output_key: response.generations[0][0].text}

    @property
    def _chain_type(self) -> str:
        return "my_custom_chain"

In [12]:
from langchain.callbacks.stdout import StdOutCallbackHandler
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate


chain = MyCustomChain(
    prompt=PromptTemplate.from_template("tell us a joke about {topic}"),
    llm=ChatOpenAI(),
)

# run 最终调用 __call__
# 源码 return self(kwargs, callbacks=callbacks, tags=tags)[_output_key]
chain.run({"topic": "callbacks"}, callbacks=[StdOutCallbackHandler()])



> Entering new  chain...
自定义chain: prompt_value=text='tell us a joke about callbacks'
自定义chain: response=generations=[[ChatGeneration(text='Why did the callback function feel lonely? Because it never gets called back!', generation_info=None, message=AIMessage(content='Why did the callback function feel lonely? Because it never gets called back!', additional_kwargs={}, example=False))]] llm_output={'token_usage': {'prompt_tokens': 14, 'completion_tokens': 15, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo'} run=RunInfo(run_id=UUID('224a4b3b-89c3-43fa-816e-3c78441674e5'))
Log something about this run
> Finished chain.


'Why did the callback function feel lonely? Because it never gets called back!'

In [13]:
### 4.debugging chains
# 将 verbose 设置为 True 将在运行 Chain 对象时打印一些内部状态。
# XXXChain(llm=llm or chat, prompt=prompt, memory=xx, verbose=True)

In [ ]:
### 5.Loading from LangChainHub
